In [1]:
from datetime import date
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd 

BASE_URL = "https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/katowice"
DOMAIN = "https://www.otodom.pl"

def get_total_pages_via_json(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Błąd: {response.status_code}")
        return 0
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    script_tag = soup.find("script", {"id": "__NEXT_DATA__"})
    if not script_tag:
        print("Nie znaleziono sekcji JSON '__NEXT_DATA__'.")
        return 0
    
    try:
        data = json.loads(script_tag.string)
        total_pages = data["props"]["pageProps"]["tracking"]["listing"]["page_count"]
        return total_pages
    except (KeyError, TypeError, json.JSONDecodeError) as e:
        print(f"Błąd podczas parsowania JSON: {e}")
        return 0




def get_page_links(url, domain):
    headers = {"User-Agent": "Mozilla/5.0"}  
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Błąd: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    links = soup.find_all("a", href=True)
    filtered_links = []
    
    for link in links:
        href = link['href']
        if href.startswith("/pl/oferta/"):
            filtered_links.append(domain + href)
    
    return list(set(filtered_links))

def save_links_to_excel(links, filename):
    df = pd.DataFrame(links, columns=["Link"])  
    df.to_excel(filename, index=False, engine='openpyxl') 

def otodom():
    total_pages = get_total_pages_via_json(BASE_URL)
    print(total_pages)
    if total_pages == 0:
        print("Nie udało się pobrać liczby stron.")
        return
    
    print(f"Liczba stron: {total_pages}")
    all_links = []
    
    for page in range(1, total_pages + 1):
        url = f"{BASE_URL}?page={page}"
        print(f"Pobieram linki z strony {page}...")
        links = get_page_links(url, DOMAIN)
        all_links.extend(links)
    
    all_links = list(set(all_links))
    for link in all_links:
        print(link)
    
    data = date.today()
    save_links_to_excel(all_links,  rf"C:\Users\PC\Desktop\scrapping_danych\otodom\otodom_links{data}.xlsx")
    print(f"Znaleziono {len(all_links)} unikalnych linków, zapisano je w pliku Excel.")

def print_html(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    print(response.text)

if __name__ == "__main__":
    otodom()

149
Liczba stron: 149
Pobieram linki z strony 1...
Pobieram linki z strony 2...
Pobieram linki z strony 3...
Pobieram linki z strony 4...
Pobieram linki z strony 5...
Pobieram linki z strony 6...
Pobieram linki z strony 7...
Pobieram linki z strony 8...
Pobieram linki z strony 9...
Pobieram linki z strony 10...
Pobieram linki z strony 11...
Pobieram linki z strony 12...
Pobieram linki z strony 13...
Pobieram linki z strony 14...
Pobieram linki z strony 15...
Pobieram linki z strony 16...
Pobieram linki z strony 17...
Pobieram linki z strony 18...
Pobieram linki z strony 19...
Pobieram linki z strony 20...
Pobieram linki z strony 21...
Pobieram linki z strony 22...
Pobieram linki z strony 23...
Pobieram linki z strony 24...
Pobieram linki z strony 25...
Pobieram linki z strony 26...
Pobieram linki z strony 27...
Pobieram linki z strony 28...
Pobieram linki z strony 29...
Pobieram linki z strony 30...
Pobieram linki z strony 31...
Pobieram linki z strony 32...
Pobieram linki z strony 33.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

data = date.today()
urls = pd.read_excel(rf'C:\Users\PC\Desktop\scrapping_danych\otodom\otodom_links{data}.xlsx')
urls_list = urls['Link'].tolist()
random_urls = random.sample(urls_list, 100) if len(urls_list) >= 50 else urls_list

def save_page_html(url):
    """Pobiera stronę i zapisuje jej HTML do słownika z danymi."""
    headers = {"User-Agent": "Mozilla/5.0"}  
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Błąd: {response.status_code}")
        return None
    soup = BeautifulSoup(response.text, 'html.parser')
    def safe_find_text(element):
        if element:
            return element.text.strip()
        return None

    price = safe_find_text(soup.find('strong', {'aria-label': 'Cena'}))

    price_per_m2 = safe_find_text(soup.find('div', {'aria-label': 'Cena za metr kwadratowy'}))

    location = safe_find_text(soup.find('a', {'href': '#map'}))
    room = safe_find_text(soup.find_all('div', class_='css-1ftqasz')[1] if len(soup.find_all('div', class_='css-1ftqasz')) > 1 else None)
    size = safe_find_text(soup.find_all('div', class_='css-1ftqasz')[0] if len(soup.find_all('div', class_='css-1ftqasz')) > 0 else None)

    data_list = soup.find_all('div', class_='css-t7cajz e15n0fyo1')
    property_info = {}

    for item in data_list:
        text = item.text.strip()
        if ':' in text:
            param, value = text.split(":", 1)
            property_info[param.strip()] = value.strip()

    property_info['Cena'] = price
    property_info['Cena za m²'] = price_per_m2
    property_info['Lokalizacja'] = location
    property_info['Liczba pokoi'] = room
    property_info['Wielkość'] = size
    property_info['URL'] = url  

    if not price or not price_per_m2 or not location:
        print(f"Brak danych na stronie: {url}, pomijamy.")
        return None

    return property_info


def scrape_multiple_urls(urls):
    """Przetwarza wiele URL i zapisuje dane do DataFrame."""
    all_data = []

    for url in urls:
        data = save_page_html(url)
        if data:
            all_data.append(data)
        else:
            print(f"Nie udało się przetworzyć URL: {url}")
    df = pd.DataFrame(all_data)
    
    return df

df = scrape_multiple_urls(urls_list)
df.to_excel(rf'C:\Users\PC\Desktop\scrapping_danych\otodom\otodom_data_{data}.xlsx', index=False)


Brak danych na stronie: https://www.otodom.pl/pl/oferta/kozielska-park-mieszkanie-2-pok-4-12-ID4pBu8, pomijamy.
Nie udało się przetworzyć URL: https://www.otodom.pl/pl/oferta/kozielska-park-mieszkanie-2-pok-4-12-ID4pBu8
Brak danych na stronie: https://www.otodom.pl/pl/oferta/barbary-19-noworoczna-promocja-m41-ID4rg8D, pomijamy.
Nie udało się przetworzyć URL: https://www.otodom.pl/pl/oferta/barbary-19-noworoczna-promocja-m41-ID4rg8D
Brak danych na stronie: https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-61m2-balkon-ID.4ulMy, pomijamy.
Nie udało się przetworzyć URL: https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-61m2-balkon-ID.4ulMy
Brak danych na stronie: https://www.otodom.pl/pl/oferta/1-pokojowe-mieszkanie-33m2-balkon-ID4ungo, pomijamy.
Nie udało się przetworzyć URL: https://www.otodom.pl/pl/oferta/1-pokojowe-mieszkanie-33m2-balkon-ID4ungo
Brak danych na stronie: https://www.otodom.pl/pl/oferta/kozielska-park-mieszkanie-2-pok-3-6-ID4pBtq, pomijamy.
Nie udało się przetworz

In [4]:
df = pd.read_excel(rf'C:\Users\PC\Desktop\scrapping_danych\otodom\otodom_data_{data}.xlsx')

In [5]:
df = df[['Cena', 'Cena za m²', 'Lokalizacja', 'Wielkość', 'Liczba pokoi', 'Piętro', 'Rodzaj zabudowy', 'Rok budowy', 'Materiał budynku', 'Typ ogłoszeniodawcy', 'URL', 'Forma własności', 'Informacje dodatkowe', 'Czynsz', 'Dostępne od', 'Stan wykończenia', 'Rynek', 'Winda', 'Bezpieczeństwo', 'Zabezpieczenia', 'Ogrzewanie']]

In [7]:
import pandas as pd
dzielnice = [
    'Śródmieście', 'Koszutka', 'Bogucice', 'Os. Paderewskiego - Muchowiec',
    'Załęże', 'Osiedle Wincentego Witosa', 'Osiedle Tysiąclecia', 'Dąb',
    'Wełnowiec-Józefowiec', 'Ligota-Panewniki', 'Brynów-Osiedle Zgrzebnioka',
    'Załęska Hałda-Brynów', 'Zawodzie', 'Dąbrówka Mała', 'Szopienice-Burowiec',
    'Janów-Nikiszowiec', 'Giszowiec', 'Murcki', 'Piotrowice-Ochojec',
    'Zarzecze', 'Kostuchna', 'Podlesie', 'Centrum', 'Józefowiec', 'Brynów', 'Ligota', 'Piotrowice', 'Nikiszowiec', 'Ochojec', 'Szopienice'
]

dzielnice_set = set(dzielnice)
def is_ulica(part):
    return part.startswith('ul.') or part.startswith('Aleja') or '/' in part

def process_reversed_lokalizacja(lokalizacja):
    parts = [part.strip() for part in lokalizacja.split(',')]
    
    wojewodztwo = miasto = dzielnica = ulica = None

    if len(parts) >= 4 and is_ulica(parts[0]):
        ulica = parts[0]
        dzielnica = parts[1] if parts[1] in dzielnice_set else "brak informacji"
        miasto = parts[2]
        wojewodztwo = parts[3]

    elif len(parts) >= 3 and parts[0] in dzielnice_set:
        dzielnica = parts[0]
        miasto = parts[1]
        wojewodztwo = parts[2]

    elif len(parts) >= 2 and parts[0] == "Katowice":
        miasto = parts[0]
        wojewodztwo = parts[1]

    return wojewodztwo, miasto, dzielnica, ulica

df[['województwo', 'miasto', 'dzielnica', 'ulica']] = df['Lokalizacja'].apply(process_reversed_lokalizacja).apply(pd.Series)
df = df.dropna(subset=['województwo'])
df = df.reset_index(drop=True)

In [9]:
def clean_and_convert(column, to_remove):
    return pd.to_numeric(
        df[column]
        .str.replace(to_remove, '', regex=True) 
        .str.extract(r'(\d{1,3}(?:[\s,]?\d{3})*(?:[.,]\d+)?)')[0]  
        .str.replace(r'[^\d,.]', '', regex=True) 
        .str.replace(r',', '.', regex=True)  
        .str.replace(r'\s+', '', regex=True) 
    )
df['Cena'] = clean_and_convert('Cena', 'zł')
df['Cena za m²'] = clean_and_convert('Cena za m²', 'zł/m²')
df['Wielkość'] = clean_and_convert('Wielkość', 'm²')
df['Czynsz'] = clean_and_convert('Czynsz', 'zł')


In [11]:
import re
def process_pietro(value):
    try:
        if not value or pd.isna(value) or value.strip() == '':
            return None, None
        value = value.lower()
        value = re.sub(r'[^0-9/parter]', '', value)
        if 'parter' in value:
            value = value.replace('parter', '0')

        if '/' in value:
            parts = value.split('/')
            pietro = int(parts[0])  
            liczba_pieter = int(parts[1])
        else:
            pietro = int(value)
            liczba_pieter = None  

        return pietro, liczba_pieter
    except (ValueError, IndexError):
        return None, None

df[['Piętro', 'Liczba pięter']] = df['Piętro'].apply(process_pietro).apply(pd.Series)

In [12]:
def extract_number(value):
    numbers = re.findall(r'\d+', str(value))
    if numbers:
        return int(numbers[0])  
    return None  

df['Liczba pokoi'] = df['Liczba pokoi'].apply(extract_number)


In [13]:
important_words = ['balkon', 'piwnica', 'garaż', 'miejsce parkingowe', 'taras', 'pom. użytkowe']
for word in important_words:
    df[word] = df['Informacje dodatkowe'].apply(lambda x: 1 if word in x else 0)

           Cena  Cena za m²  \
0      326000.0        7143   
1     1220000.0       15062   
2      777000.0        9107   
3      726408.0       11800   
4      599000.0        8509   
...         ...         ...   
2342  1036000.0       13632   
2343   732193.0       12950   
2344   598000.0       13907   
2345   534000.0        8236   
2346   751292.0       11865   

                                            Lokalizacja  Wielkość  \
0     ul. Jaworowa, Piotrowice-Ochojec, Katowice, śl...     45.64   
1     ul. gen. Władysława Sikorskiego, Osiedle Pader...     81.00   
2     ul. Juliusza Słowackiego, Śródmieście, Katowic...     85.32   
3     ul. Rodańska, Osiedle Paderewskiego-Muchowiec,...     61.56   
4     ul. Zofii Nałkowskiej, Janów-Nikiszowiec, Kato...     70.40   
...                                                 ...       ...   
2342                     Śródmieście, Katowice, śląskie     76.00   
2343  ul. gen. Kazimierza Pułaskiego 23, Osiedle Pad...     56.54   
2344  

In [14]:
df['Ogrzewanie_rodzaj'] = df['Ogrzewanie']
df['Ogrzewanie'] = df['Ogrzewanie'].apply(lambda x: 0 if pd.isna(x) or x == 'brak informacji' else 1)

In [15]:
df['ochrona'] = df['Bezpieczeństwo'].apply(lambda x: 0 if pd.isna(x) or x == 'brak informacji' else 1)

In [16]:
df['Winda'] = df['Winda'].apply(lambda x: 0 if x == 'nie' else 1)

In [18]:
df.to_excel(rf'C:\Users\PC\Desktop\scrapping_danych\otodom\data_archive\otodom_data_preprocess_{data}.xlsx', index=False)